In [1]:
# !apt-get update
# !apt-get install -y libsm6 libxext6 libxrender-dev
# !pip install opencv-python

In [1]:
#!pip install openmim
#!mim install mmdet

In [15]:
# 모듈 import

from mmcv import Config
from mmseg.apis import train_segmentor
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor

In [16]:
# config file 들고오기
# cfg = Config.fromfile('./configs/_base_/my_seg/upernet_swin_base_wind12_512.py')
cfg = Config.fromfile('./configs/_base_/my_seg/fcn_hr48_512x512_160k_ade20k.py')

cfg.data.samples_per_gpu = 12
cfg.data.workers_per_gpu = 4

cfg.seed = 2021
cfg.gpu_ids = [0]

cfg.work_dir = './work_dirs/fcn_hr48'

cfg.model.decode_head.num_classes=11
# for swin
# cfg.model.auxiliary_head.num_classes=11

# image_size 변환 - default : (1024, 1024), 적용시 (2048, 2048)
# cfg.train_pipeline[2].img_scale=(2048,2048)
# cfg.test_pipeline[1].img_scale=(2048,2048)
# cfg.val_pipeline[2].img_scale=(2048,2048)

# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=25, interval=1)
cfg.runner = dict(type='EpochBasedRunner', max_epochs=30)
cfg.lr_config =  dict(
    policy='CosineAnnealing',
    warmup='linear',
    warmup_iters=200,
    warmup_ratio=0.001,
    min_lr_ratio=1e-4,
)

# wandb log설정
cfg.log_config = dict(
    interval=100,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='EpochWandbLogger',
        init_kwargs=dict(
            project='Swin_seg_mmseg',
            name='hr_net_no_change'))
])
cfg.workflow = [('train', 1),('val',1)]
cfg.evaluation = dict(interval=1, by_epoch=True, metric='mIoU')
# cfg.norm_cfg = dict(type='BN', requires_grad=True)

In [17]:
# build_dataset
datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val_loss)] # 

2021-10-21 12:46:58,040 - mmseg - INFO - Loaded 2617 images
2021-10-21 12:46:58,052 - mmseg - INFO - Loaded 655 images


In [18]:
datasets[0]

In [19]:
# 모델 build 및 pretrained network 불러오기
model = build_segmentor(cfg.model)
model.init_weights()

2021-10-21 12:46:59,465 - mmseg - INFO - initialize HRNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://msra/hrnetv2_w48'}
2021-10-21 12:46:59,466 - mmcv - INFO - load model from: open-mmlab://msra/hrnetv2_w48
2021-10-21 12:46:59,467 - mmcv - INFO - Use load_from_openmmlab loader
2021-10-21 12:47:00,151 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: incre_modules.0.0.conv1.weight, incre_modules.0.0.bn1.weight, incre_modules.0.0.bn1.bias, incre_modules.0.0.bn1.running_mean, incre_modules.0.0.bn1.running_var, incre_modules.0.0.bn1.num_batches_tracked, incre_modules.0.0.conv2.weight, incre_modules.0.0.bn2.weight, incre_modules.0.0.bn2.bias, incre_modules.0.0.bn2.running_mean, incre_modules.0.0.bn2.running_var, incre_modules.0.0.bn2.num_batches_tracked, incre_modules.0.0.conv3.weight, incre_modules.0.0.bn3.weight, incre_modules.0.0.bn3.bias, incre_modules.0.0.bn3.running_mean, incre_modules.0.0.bn3.running_va

In [ ]:
# 모델 학습
train_segmentor(model, datasets, cfg, distributed=False, validate=True)

2021-10-21 12:47:01,851 - mmseg - INFO - Loaded 655 images
2021-10-21 12:47:01,852 - mmseg - INFO - Start running, host: root@65f88b0d31f6, work_dir: /opt/ml/segmentation/github_workspace/semantic-segmentation-level2-cv-07/mmsegmentation/work_dirs/fcn_hr48
2021-10-21 12:47:01,853 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) EpochWandbLogger                   
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) EpochWandbLogger                   
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook     

wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2021-10-21 12:48:14,656 - mmseg - INFO - Epoch [1][100/327]	lr: 4.955e-03, eta: 1:39:49, time: 0.617, data_time: 0.030, memory: 11719, decode.loss_ce: 1.0838, decode.acc_seg: 55.5891, loss: 1.0838
2021-10-21 12:49:13,736 - mmseg - INFO - Epoch [1][200/327]	lr: 9.950e-03, eta: 1:36:42, time: 0.591, data_time: 0.005, memory: 11719, decode.loss_ce: 0.6267, decode.acc_seg: 68.2082, loss: 0.6267
